# Práctica NLP
Hecha por Rubén Cerezo Cuesta
En esta práctica, vamos a hacer dos modelos de NLP a partir de un dataset de Amazon formado por reseñas de videojuegos. Con esto, esperamos conseguir un análisis de sentimiento. Tras entrenar estos modelos, podremos conseguir una herramienta que catalogue reviews en función de cómo de positivas son. 
Este trabajo está dividido en 4 fases:
- Exploración de datos
- **Preprocesamiento de datos**
- división Train/Test y entrenamiento
- Métricas y conclusiones
En este caso, el corpus elegido, de Amazon, incluye reviews de videojuegos, simplemente por ser un tema que conozco y que he pensado que podría resultar fácil a la hora de reconocer el producto con el que trabajo. 

## Preprocesado
En este notebook, vamos a realizar los siguientes pasos : 
- Cargamos el archivo .csv donde se encuentra el corpus
-El alumno preparará una etapa de preprocesado de reviews que permita adecuar
el formato de las mismas a uno más adecuado. Será la etapa previa al entrenamiento del
modelo de sentimiento.
Todo el preprocesado deberá incluirse en una función de Python que contenga
todo el procesado de texto. Esta función puede (es recomendable) contener otras funciones
que realicen tareas más concretas (eliminar stopwords, eliminar signos de puntuación, etc

In [2]:
!pip install unicodedata


ERROR: Could not find a version that satisfies the requirement unicodedata (from versions: none)
ERROR: No matching distribution found for unicodedata


In [2]:
#Cargamos el archivo .csv 
import pandas as pd

path = r'C:\Users\rammu\Documents\projects\NLP_04\práctica\reviews_Video_Games_5_balanced.csv'

df = pd.read_csv(path)
df.head()


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3DLWSSO2LVC1E,B000FDOTIQ,Kora,"[4, 6]",Final Fantasy VII for the ps1 was (and still i...,2,More like a movie than a game.,1164844800,"11 30, 2006"
1,AJIDIVBILJKO0,B002DY9KHU,MekoRush,"[0, 0]",Yeah it's ok. it was easy to get use to the co...,3,"Sneakly reviewing this game, it's ok",1334534400,"04 16, 2012"
2,A3PZ4AXTY9J1DZ,B000R39GPA,Jason Ralsky,"[2, 2]",Star Wars: The Force Unleashed (SW:TFU) at lau...,3,Misses the mark of past Jedi games just barely,1287964800,"10 25, 2010"
3,AW3BDUZUFZMFX,B0000A92KZ,Joshua T. Garcia,"[1, 2]",This review may contain minor spoilers. It was...,2,The Fall of Max Payne,1318118400,"10 9, 2011"
4,A1GBZD75UNMD2B,B001TOQ8R0,"Adgear ""Derebu""","[8, 9]",I'm having the same problems as everyone else ...,1,Don't buy this game!!,1269388800,"03 24, 2010"


Como vemos, el dataset vuelve a estar con todas las columnas, ya que el notebook anterior sólo lo utilizamos para analizar, pero el preprocesado real va a ocurrir en este notebook

In [3]:
#Eliminamos columnas innecesarias
df_balanced = df.drop(columns=['reviewerID', 'asin', 'reviewerName', 'helpful', 'unixReviewTime', 'reviewTime', 'summary'])
df_balanced.head()


,reviewText,overall
0,Final Fantasy VII for the ps1 was (and still i...,2
1,Yeah it's ok. it was easy to get use to the co...,3
2,Star Wars: The Force Unleashed (SW:TFU) at lau...,3
3,This review may contain minor spoilers. It was...,2
4,I'm having the same problems as everyone else ...,1


Como dijimos en el notebook anterior, parte del preprocesado de los datos incluiría eliminar de nuestro corpus aquellos nombres que puedan pertenecer a franquicias. Para ello, lo primero que vamos a hacer es visualizar todo aquello que Spacy reconoce como entidades y cuáles son sus categorías. 

In [4]:
import spacy 
#visualizar categorías de entidades en el NER DE spacy
nlp = spacy.load("en_core_web_sm", disable=["parser"])

from collections import defaultdict

entity_examples = defaultdict(list)

for doc in nlp.pipe(df_balanced["reviewText"].astype(str), batch_size=50):
    for ent in doc.ents:
        if len(entity_examples[ent.label_]) < 10:   # muestra hasta 10 ejemplos por categoría
            entity_examples[ent.label_].append(ent.text)

# Mostrar resultados
for label, examples in entity_examples.items():
    print(f"\n=== {label} ===")
    for e in examples:
        print("  -", e)



=== ORG ===
  - Final Fantasy VII
  - RPG
  - WANT
  - Final Fantasy VII
  - Advent Children
  - DBZ
  - SW
  - TFU
  - Starkiller
  - holocron

=== CARDINAL ===
  - 3
  - 1
  - 2
  - 2
  - 3
  - 1
  - 2
  - 5
  - 4
  - 4X

=== TIME ===
  - every two minutes or so
  - about 6 hours
  - an hour
  - minutes
  - 15 hours
  - late night
  - late at night
  - only several hours
  - only a few hours
  - this evening

=== DATE ===
  - the year
  - the years
  - 2011
  - a few weeks
  - 30fps
  - 1080p
  - Tomorrow
  - 10 years
  - 194223
  - 194224

=== MONEY ===
  - 20(what
  - 20
  - 30
  - 9.99
  - another $50
  - 60
  - another $50
  - 90
  - 34;Free
  - 60

=== PERSON ===
  - Jedi Academy
  - Lucas Arts
  - 720p
  - Max Payne
  - Remedy
  - Max Payne
  - Remedy
  - Max
  - Max Payne
  - Max Payne

=== ORDINAL ===
  - first
  - second
  - third
  - first
  - first
  - first
  - third
  - first
  - first
  - first

=== EVENT ===
  - the Dark Side / Light Side
  - World v World v World
  -

### Qué hacer con las entidades y franquicias: 
Visto esto, eliminaremos las siguientes categorías:
1. ORG
2. PERSON
3. GPE
4. PRODUCT
5. LOC
6. WORK_OF_ART
7. FAC
8. LAW

Porque claramente, muestran entidades que no aportan al nivel de sentimiento, sino que se asociarían con el producto del que hablamos (por ejemplo, es claro que mencionen Star Wars en una review de un videojuego de esta franquicia, y también LucasArts, la compañía que lo produce)



In [5]:
def remove_entities(text):
    """
    Recibe un string, aplica NER con spaCy y elimina las entidades
    pertenecientes a REMOVE_LABELS. Devuelve un string limpio.
    """
    import spacy

    # Cargar spaCy solo una vez sería mejor, pero aquí respetamos tu estilo
    nlp = spacy.load("en_core_web_sm", disable=["parser"])

    REMOVE_LABELS = {
        "ORG", "PERSON", "GPE", "PRODUCT",
        "LOC", "WORK_OF_ART", "FAC", "LAW"
    }

    # Convertir a string (por seguridad)
    text = str(text)

    # Procesamos el texto
    doc = nlp(text)

    # Identificar spans a eliminar
    spans_to_remove = []
    for ent in doc.ents:
        if ent.label_ in REMOVE_LABELS:
            spans_to_remove.append((ent.start_char, ent.end_char))

    # Si no hay entidades → devolvemos el texto normalizado
    if not spans_to_remove:
        return " ".join(text.split())

    # Construir el texto sin las entidades
    cleaned_parts = []
    last_end = 0

    for start, end in spans_to_remove:
        cleaned_parts.append(text[last_end:start])
        last_end = end

    cleaned_parts.append(text[last_end:])
    cleaned_text = "".join(cleaned_parts)

    # Normalizar espacios antes de devolver
    return " ".join(cleaned_text.split())
print (remove_entities(df_balanced))


reviewText overall 0 Final Fantasy VII for the ps1 was (and still i... 2 1 Yeah it's ok. it was easy to get use to the co... 3 2 Star Wars: The Force Unleashed (:) at lau... 3 3 This review may contain minor spoilers. It was... 2 4 I'm having the same problems as everyone else ... 1 ... ... ... 4995 Besides better graphics and effects the overal... 5 4996 i picked this up in a bargain bin at a local v... 1 4997 I played the first Sniper Elite and my beef wa... 4 4998 SimCity 4 is the latest in the long-survived S... 4 4999 THESE GAMES ARE STUPID, MAKE NO SENSE AT ALL. ... 1 [5000 rows x 2 columns]


In [6]:
from funciones_preprocesamiento import preprocess_text, remove_stopwords, lemmatize_text
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = set(STOP_WORDS)

# Preprocesamos, como en el anterior notebook, las review, quitando puntuación, pasando a minúsculas 
df_balanced= preprocess_text(df_balanced)   
# Quitamos stopwords

df_balanced["reviewText"] = df_balanced["reviewText"].apply(lambda text: remove_stopwords(text, stopwords))
df_balanced["reviewText"] = df_balanced["reviewText"].apply(lemmatize_text)
df_balanced.head()

,reviewText,overall
0,final fantasy vii ps1 consider good rpgs creat...,2
1,yeah ok easy use control ol story you ll play ...,3
2,star war force unleash swtfu launch probably h...,3
3,review contain minor spoiler write specificall...,2
4,I m have problem get disconnected huge waste m...,1


In [7]:
#Tras todo el preprocesado, hacemos una tokenización final:
def tokenize(text):
    return text.split()
# Aplicamos la tokenización
df_balanced["reviewText"] = df_balanced["reviewText"].apply(tokenize)
df_balanced.head()


,reviewText,overall
0,"[final, fantasy, vii, ps1, consider, good, rpg...",2
1,"[yeah, ok, easy, use, control, ol, story, you,...",3
2,"[star, war, force, unleash, swtfu, launch, pro...",3
3,"[review, contain, minor, spoiler, write, speci...",2
4,"[I, m, have, problem, get, disconnected, huge,...",1


In [8]:
import unicodedata,re
#mostramos una review original y su versión preprocesada
t = df['reviewText'].iloc[0]
t_processed = df_balanced['reviewText'].iloc[0]
print("Original review:\n", t)
print("\nPreprocessed review:\n", t_processed)




Original review:
 Final Fantasy VII for the ps1 was (and still is) considered one of the best RPG's ever created. The same cannot be said for the spin off of a side quest character that tries too hard to be hard core and ends up being mistaken for a Devil May Cry clone.The good news:-You can customize your weapons to cause maximum damage.-It's kind of neat to see final fantasy VII characters in 3-D with voices.The bad news:-This game is horribly paced. You can't get into the game play because there's a cut scene every two minutes or so. Most of them have nothing to do with where you are fighting so they seem terribly misplaced and you just don't really care enough about the characters to WANT to watch them. This game is way too concerned about being an animated movie rather than an actual game.-The aiming system is shot and you cannot lock onto enemies the way you should be able to. If this had been fixed, this could have at least been an average game.-The same goes for the camera, you

In [9]:
#Por último, guardamos el dataframe preprocesado en un nuevo archivo .csv
df_balanced.to_csv('reviews_Video_Games_5_balanced_preprocessed.csv', index=False)


# Creación de una función que englobe todo este proceso: 

Para finalizar, y después de haber ido haciendo todo el proceso poco a poco, vamos a crear una función que haga todos los pasos anteriores para así evitar tener que revisar código en caso de que, en futuros entrenamientos haya que volver a entrenar el modelo con nuevas entradas en el corpus: 



In [10]:
print("dtype reviewText:", df_balanced['reviewText'].dtype)
print("nulos reviewText:", df_balanced['reviewText'].isna().sum())
print("Ejemplos nulos / None:")
print(df_balanced[df_balanced['reviewText'].isna()].head(10))

dtype reviewText: object
nulos reviewText: 0
Ejemplos nulos / None:
Empty DataFrame
Columns: [reviewText, overall]
Index: []


In [11]:
import pandas as pd

from funciones_preprocesamiento import  preprocess_text, remove_stopwords, lemmatize_text
input_path = r'C:\Users\rammu\Documents\projects\NLP_04\práctica\reviews_Video_Games_5_balanced.csv'
output_path = r'C:\Users\rammu\Documents\projects\NLP_04\práctica\reviews_Video_Games_5_balanced_preprocessed.csv'
def preprocess_complete (input_path = input_path , output_path= output_path):
    """
    Esta función hace el preprocesamiento completo del texto, utilizando las funciones ya definidas antes. Como parámetros, usa el path donde el .csv está guardado, y el path donde 
    queremos guardar nuestro dataset ya preprocesado 
    
    :param input_path: como input_path mantenemos la ruta donde está guardado el archivo
    :param output_path: Como output_path ponemos la ruta donde queremos guardar el archivo, normalmente la carpeta raíz del proyecto, para poder mantener coherencia con el siguiente notebook

    """
    #Cargamos el dataset
    df = pd.read_csv(input_path)
    #Eliminamos las columnas innecesarias
    df_balanced = df.drop(columns=['reviewerID', 'asin', 'reviewerName', 'helpful', 'unixReviewTime', 'reviewTime', 'summary'])
    #Aplicamos las funciones que hemos creado para este preprocesamiento:
    #Remove entities para eliminar títulos de videojuegos
    #preprocess_text para eliminar mayúsculas y puntuación
    # remove_stopwords para eliminar palabras con poca carga semántica.
    #lemmatize_text para mantener sólo las raíces de las palabras
    df_balanced["reviewText"] = (
    df_balanced["reviewText"]
        .fillna("")
        .astype(str)
        .apply(remove_entities)
        .apply(preprocess_text)
        .apply(lambda t: remove_stopwords(t, stopwords))
        .apply(lemmatize_text)
)
    #Por último, guardamos nuestro corpus ya preprocesado en el "output_path definido" 
    df_balanced.to_csv(output_path, index=False)
    
    return df_balanced 

preprocess_complete()
df_balanced.head

TypeError: string indices must be integers, not 'str'

Creamos un segundo csv en el que cambiamos las reseñas a valores 0-1 (negativo y positivo) para simplificar el modelo y conseguir una mayor accuracy. 
Sin embargo, he hecho 2 notebooks distintos que usaré para comparar los resultados entre modelos entrenados con distintos corpus. 
Para esto, vamos a cambiar las reseñas puntuadas con 1 y 2 a un valor 0 (negativo) y las reseñas puntuadas con 4 y 5 a un valor 1 (positivo). 
Tras hacer una pequeña búsqueda en Google, he decidido desechar las reseñas evaluadas en 3 dado que, al ser neutras, podrian aportar ruido y reducir la precisión del modelo 

In [ ]:
path= r'C:\Users\rammu\Documents\projects\NLP_04\práctica\reviews_Video_Games_5_balanced_preprocessed.csv'
def change_overall(corpus, path):
    """
    Esta función transforma puntuaciones 1-5 en sentimiento binario:
    - 1 y 2 => 0 (negativo)
    - 4 y 5 => 1 (positivo)
    Elimina las puntuaciones neutras (3).

    Recibe la ruta del archivo .csv y un dataframe (no usado) y devuelve
    el dataframe modificado. Además, guarda un nuevo .csv.
    """

    # Cargar dataset desde la ruta
    corpus = pd.read_csv(path)

    # Eliminar reseñas con puntuación = 3
    corpus = corpus[corpus['overall'] != 3].copy()

    # Aplicar mapeo a 0 (negativo) o 1 (positivo)
    corpus['overall'] = corpus['overall'].apply(lambda x: 1 if x >= 4 else 0)

    # Guardar nuevo archivo
    output_path = 'reviews_Video_Games_5_balanced_preprocessed_0-1.csv'
    corpus.to_csv(output_path, index=False)

    return corpus
#aplicamos la función
df_final= change_overall(df_balanced, path)
#comprobamos el resultado
df_final = pd.read_csv('reviews_Video_Games_5_balanced_preprocessed_0-1.csv')
df_final['overall'].value_counts()

overall
0    2000
1    2000
Name: count, dtype: int64